In [4]:
import numpy as np
from solver_funcs import F_operator, mu_from_params
import matplotlib.pyplot as plt
from utils import noise_addition_image
from scipy.special import erf
from emcee import EnsembleSampler

## Define global variables

sigma = 5e-6

param_lims_dict = {
    0: (0.4, 1.2),
    1: (2.38, 2.52),
    2: (2.3, 3.1),
    3: (-0.04, 0.04),
    4: (-0.08, 0.08),
    5: (2.68, 2.82),
    6: (-0.08, 0.08),
    7: (2.68, 2.82),
    8: (1, 5),
    9: (-0.2, 0.2),
    10: (0.01, 0.11),
    11: (-0.11, -0.01),
    12: (0.5, 0.7),
    13: (0, 0.2),
    14: (0.45, 1.15),
    15: (8, 16),
    16: (0, 2),
    17: (0.2, 0.4),
    18: (0.2, 0.4),
    19: (0.04, 0.16),
    20: (0.55, 0.95),
    21: (0.55, 0.95),
    22: (13, 17),
    23: (13, 17),
    24: (0.06, 0.1),
    25: (0.06, 0.1),
    26: (0, 146200),
    27: (0, 250000),
    28: (0, 19600),
    29: (0, 19600),
    30: (0, 19600),
    31: (0, 250000),
    32: (-np.pi / 12, np.pi / 12),
}

prior_params = {
    0: (73100, 46900),
    1: (125000, 50000),
    2: (9800, 3340),
    3: (9800, 3340),
    4: (9800, 3340),
    5: (125000, 50000),
}


step_stdev = np.zeros(33)

for i in range(33):
    step_stdev[i] = param_lims_dict[i][1] - param_lims_dict[i][0]

## Functions
    
def sample_mean_params(param_lims):
    params_list = []
    for i in range(len(param_lims)):
        params_list.append((param_lims[i][1] - param_lims[i][0]) / 2 + param_lims[i][0])
    return np.array(params_list)
    
## Target

mu_params = sample_mean_params(param_lims_dict)
mu_params[26:32] = mu_params[26:32] * 0.2
mu = mu_from_params(mu_params)
mu, y = F_operator(mu_params)
y_meas = noise_addition_image(y, noise_level=sigma)
np.save("measurement_5em6_0dot2mean.npy", y_meas)


def calc_gn_cfd(x):
    return erf(x / np.sqrt(2)) / 2 + 0.5


def calc_norm_ct(mean, std):
    return calc_gn_cfd(mean / std) - calc_gn_cfd(-mean / std)


prior_norm_ct = dict()

for key in prior_norm_ct.keys():
    prior_norm_ct[key] = calc_norm_ct(prior_params[key][0], prior_params[key][1])


def calc_step(factor):
    return np.random.normal(scale=step_stdev) * factor


def check_inside_prior(param_list, param_lims):
    for i in range(len(param_list)):
        if param_list[i] < param_lims[i][0] or param_list[i] > param_lims[i][1]:
            return False
    return True


def calc_logprob_prior(param_list, prior_params):
    logprob = 0
    for i in range(len(prior_params)):
        logprob += (
            -0.5
            * (param_list[26 + i] - prior_params[i][0]) ** 2
            / prior_params[i][1] ** 2
        ) / prior_norm_ct[i]
    return logprob


def calc_loglikelihood(param_list, meas_y, sigma):
    mu, y_cur = F_operator(param_list)
    mask = mu != 100
    mask = mask.astype(int)
    mask = np.repeat(mask, 2, axis=3)
    y_cur = np.multiply(y_cur, mask)
    loglikelihood = -0.5 * np.sum(np.power(meas_y - y_cur, 2)) / sigma**2
    return mu, loglikelihood


def calc_logpostprob(param_list):
    mu, loglikelihood = calc_loglikelihood(param_list, y_meas, sigma)
    return loglikelihood + calc_logprob_prior(param_list, prior_params)


/Users/javiermurgoitioesandi/Desktop/WIP/ONH_InverseElast/ONHFenics_library_1/mu_funcs_given.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  (self.LC_y_3 - self.LC_y_2) / (self.LC_x_3 - self.LC_x_2),
/Users/javiermurgoitioesandi/Desktop/WIP/ONH_InverseElast/ONHFenics_library_1/mu_funcs_given.py:171: RuntimeWarning: divide by zero encountered in double_scalars
  (self.LC_y_4 - self.LC_y_1) / (self.LC_x_4 - self.LC_x_1),
/Users/javiermurgoitioesandi/Desktop/WIP/ONH_InverseElast/ONHFenics_library_1/mu_funcs_given.py:219: RuntimeWarning: divide by zero encountered in double_scalars
  (self.SCTL_y_3 - self.SCTL_y_2) / (self.SCTL_x_3 - self.SCTL_x_2),
/Users/javiermurgoitioesandi/Desktop/WIP/ONH_InverseElast/ONHFenics_library_1/mu_funcs_given.py:361: RuntimeWarning: divide by zero encountered in double_scalars
  (self.SCTR_y_4 - self.SCTR_y_1) / (self.SCTR_x_4 - self.SCTR_x_1),
/Users/javiermurgoitioesandi/Desktop/WIP/ONH_InverseElast/ONHFenics_library_1/mu_funcs_given

No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.548e-03 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 4.114e-18 (tol = 1.000e-10) r (rel) = 2.657e-15 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.


In [ ]:
sampler = EnsembleSampler(nwalkers, ndim, log_prob_func, blobs_dtype=dtype)


In [ ]:
sampler.run_mcmc(coords, 5000, progress=True)